In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Tutorial 4: Quantum devices
## Quantum Memory

In [ ]:
from netsquid.components import QuantumMemory
from netsquid.components.models import DepolarNoiseModel

# Depolarization rate = 1e8 Hz = 1/(10 ns)
qerror_model = DepolarNoiseModel(depolar_rate=1e8)

qmem = QuantumMemory("Tutorial memory", 
                     num_positions=10,
                     memory_noise_models=qerror_model)

In [ ]:
qubit, = ns.qubits.create_qubits(1)

ns.qubits.reduced_dm([qubit])

In [ ]:
qmem.put(qubit, positions=2)

ns.sim_run(duration=10)

qubit, = qmem.peek(positions=2)

ns.qubits.reduced_dm([qubit])

### Different noise model per memory position

In [ ]:
from netsquid.components import QuantumMemory
from netsquid.components.models import DepolarNoiseModel

qerror_model_mem_pos_0 = DepolarNoiseModel(depolar_rate=1e8)
qerror_model_mem_pos_1 = DepolarNoiseModel(depolar_rate=1e7)

qmem = QuantumMemory("Tutorial memory2", 
                     num_positions=2,
                     memory_noise_models=[qerror_model_mem_pos_0,
                                          qerror_model_mem_pos_1])

q1, q2 = ns.qubits.create_qubits(2)

qmem.put([q1, q2], positions=[0, 1])

ns.sim_run(duration=10)

q1, q2 = qmem.peek(positions=[0, 1])

print("q1 DM after 10ns in qmem:")
print(ns.qubits.reduced_dm([q1]))

print("\nq2 DM after 10ns in qmem:")
print(ns.qubits.reduced_dm([q2]))

## Quantum Processor

In [ ]:
# Depolarization rate = 1e8 Hz = 1/(10 ns)
memory_qerror_model = DepolarNoiseModel(depolar_rate=1e8)
# Depolarization probability = 10%
gate_qerror_model = DepolarNoiseModel(0.1, time_independent=True)

### Physical instructions

In [ ]:
from netsquid.components import PhysicalInstruction, INSTR_INIT, INSTR_H, INSTR_CNOT, INSTR_MEASURE

# Qubit initialization and measurement take 3ns
instr_init = PhysicalInstruction(INSTR_INIT,
                                 duration=3)

instr_meas = PhysicalInstruction(INSTR_MEASURE,
                                 duration=3)

# H gate takes 1ns, only available at mem_pos 1,2 and 3 (not available at 0), use error model of 10% depolarization probability 
instr_h = PhysicalInstruction(INSTR_H,
                              duration=1,
                              topology=[1,2,3],
                              quantum_noise_model=gate_qerror_model)

# CNOT take 2ns, only available when mem_pos 1 is control qubit and 2 or 3 are target qubit
instr_cnot = PhysicalInstruction(INSTR_CNOT,
                                 duration=2,
                                 topology=[(1, 2), (1, 3)])

### Creating a quantum processor

In [ ]:
from netsquid.components import QuantumProcessor

quantum_processor = QuantumProcessor(
    "Demo quantumProcessor",
    num_positions=4,
    mem_noise_models=memory_qerror_model,
    phys_instructions=[instr_init, instr_meas, instr_h, instr_cnot]
    )

### Quantum programs

In [ ]:
from netsquid.components import QuantumProgram

create_program = QuantumProgram(num_qubits=2)
q1, q2 = create_program.get_qubit_indices()

create_program.apply(INSTR_INIT, q1)
create_program.apply(INSTR_INIT, q2)
create_program.apply(INSTR_H, q1)
create_program.apply(INSTR_CNOT, [q1, q2])


class MeasureProgram(QuantumProgram):
    default_num_qubits = 2
    
    def program(self):
        q1, q2 = self.get_qubit_indices(2)
        # inplace=False will free the memory position after measurement
        self.apply(INSTR_MEASURE, q1, output_key="m1", inplace=False)
        self.apply(INSTR_MEASURE, q2, output_key="m2", inplace=False)
        yield self.run()

measure_program = MeasureProgram()


### Running a quantum program

In [ ]:
ns.sim_reset()

quantum_processor.execute_program(create_program, 
                                  qubit_mapping=[1,2])

ns.sim_run()

qubits = quantum_processor.peek(positions=[1,2])
print(f"{ns.sim_time()}ns Finished create program. \nDM:\n{ns.qubits.reduced_dm(qubits)}")


quantum_processor.execute_program(measure_program, 
                                  qubit_mapping=[1,2])

ns.sim_run()

m1 = measure_program.output["m1"]
m2 = measure_program.output["m2"]

print(f"\n{ns.sim_time()}ns Finished measure program with results: {m1}, {m2}")


Next: [Exercise 1](exercise1.ipynb)